In [ ]:
import redis
import pickle
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import io
import threading
import time
import traceback
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

# ---------------------- 配置参数 ----------------------
REDIS_CONFIG_PATH = "redis.conf"
RESULT_QUEUE = "function_results"
STORAGE_ROOT = r"D:\workspace\xiaoyao\data\stock_minutely_price"
THREAD_NUM = 8  # 线程数：建议设为CPU核心数
PROGRESS_INTERVAL = 10  # 进度打印间隔（秒）
IDLE_TIMEOUT = 60  # 空闲超时时间（秒）：无新任务60秒后自动退出

# ---------------------- 工具函数 ----------------------
def load_redis_config(config_path):
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Redis配置文件不存在：{config_path}")
    
    host = "localhost"
    port = 6379
    password = ""
    
    with open(config_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            if line.startswith('host='):
                host = line.split('=', 1)[1].strip()
            elif line.startswith('port='):
                try:
                    port = int(line.split('=', 1)[1].strip())
                except ValueError:
                    print(f"警告：port格式错误，使用默认值{port}")
            elif line.startswith('password='):
                password = line.split('=', 1)[1].strip()
    
    return {
        "host": host,
        "port": port,
        "password": password,
        "decode_responses": False,
        "socket_timeout": 30,
        "socket_keepalive": True
    }

# ---------------------- 动态任务处理类 ----------------------
class DynamicTaskProcessor:
    def __init__(self, redis_config):
        # 初始化存储目录
        os.makedirs(STORAGE_ROOT, exist_ok=True)
        print(f"✅ 存储目录：{STORAGE_ROOT}")
        
        # Redis连接
        self.redis = redis.Redis(** redis_config)
        self._test_redis()
        
        # 任务元信息键
        self.metadata_key = "task_metadata"
        
        # 统计信息（线程安全）
        self.stats = {
            "success": 0,
            "failed": 0,
            "start_time": time.time(),
            "last_task_time": time.time()  # 最后一次处理任务的时间
        }
        self.stats_lock = threading.Lock()
        
        # 线程池
        self.thread_pool = ThreadPoolExecutor(max_workers=THREAD_NUM)
        print(f"✅ 线程池初始化完成（{THREAD_NUM}线程）")
        
        # 控制程序退出的标志
        self.running = True
        print(f"✅ 动态任务处理器启动，将持续监听队列 '{RESULT_QUEUE}'")
        print(f"ℹ️  无新任务{IDLE_TIMEOUT}秒后将自动退出")

    def _test_redis(self):
        try:
            self.redis.ping()
            print("✅ Redis连接成功")
        except Exception as e:
            print(f"❌ Redis连接失败：{e}")
            raise SystemExit(1)

    def _update_stats(self, is_success):
        """更新统计信息并定期打印进度"""
        with self.stats_lock:
            if is_success:
                self.stats["success"] += 1
            else:
                self.stats["failed"] += 1
            
            # 更新最后任务时间
            self.stats["last_task_time"] = time.time()
            
            # 定期打印进度
            current_time = time.time()
            elapsed = current_time - self.stats["start_time"]
            
            if int(elapsed) % PROGRESS_INTERVAL == 0 and elapsed > 1:
                processed = self.stats["success"] + self.stats["failed"]
                speed = processed / elapsed if elapsed > 0 else 0
                
                print(f"📊 累计处理：{processed} 任务 | 成功：{self.stats['success']} | 失败：{self.stats['failed']} "
                      f"| 平均速度：{speed:.1f}任务/秒 | 运行时间：{int(elapsed)}秒")

    def _process_single_task(self, task_id, csv_str, trade_date, stock_code):
        """处理单个任务"""
        try:
            # 内存读取CSV
            df = pd.read_csv(
                io.StringIO(csv_str),
                encoding='utf-8',
                sep=',',
                on_bad_lines='skip',
                dtype={
                    "date": str, "stock_code": str, "time": str,
                    "open": float, "close": float, "high": float,
                    "low": float, "volume": int
                }
            )
            
            # 数据校验
            required_cols = ["date", "stock_code", "time", "open", "close", "high", "low", "volume"]
            if not all(col in df.columns for col in required_cols):
                raise ValueError(f"缺少字段：{[c for c in required_cols if c not in df.columns]}")
            
            # 转换为Arrow Table（兼容旧版本）
            schema = pa.schema([
                pa.field("date", pa.string()),
                pa.field("stock_code", pa.string()),
                pa.field("time", pa.string()),
                pa.field("open", pa.float64()),
                pa.field("close", pa.float64()),
                pa.field("high", pa.float64()),
                pa.field("low", pa.float64()),
                pa.field("volume", pa.int64())
            ])
            table = pa.Table.from_pandas(df, schema=schema)
            
            # 写入Parquet
            parquet_dir = os.path.join(STORAGE_ROOT, f"date={trade_date}", f"stock_code={stock_code}")
            os.makedirs(parquet_dir, exist_ok=True)
            parquet_path = os.path.join(parquet_dir, "data.parquet")
            
            if os.path.exists(parquet_path):
                existing_table = pq.read_table(parquet_path)
                combined_table = pa.concat_tables([existing_table, table])
                pq.write_table(combined_table, parquet_path, compression="snappy")
            else:
                pq.write_table(table, parquet_path, compression="snappy")
            
            # 清理元信息
            self.redis.hdel(self.metadata_key, task_id)
            self._update_stats(is_success=True)
            # 简化日志，避免刷屏
            if self.stats["success"] % 100 == 0:  # 每100个成功任务打印一次详细日志
                print(f"✅ 任务{task_id}处理成功（累计成功：{self.stats['success']}）")
            
        except Exception as e:
            self._update_stats(is_success=False)
            # 每10个失败任务打印一次详细日志
            if self.stats["failed"] % 10 == 0:
                print(f"❌ 任务{task_id}处理失败：{str(e)[:80]}（累计失败：{self.stats['failed']}）")

    def _check_idle_timeout(self):
        """检查是否超时无新任务"""
        with self.stats_lock:
            idle_time = time.time() - self.stats["last_task_time"]
            return idle_time >= IDLE_TIMEOUT

    def start_listening(self):
        """持续监听Redis队列，处理动态任务"""
        try:
            while self.running:
                # 检查是否超时无任务
                if self._check_idle_timeout():
                    print(f"ℹ️  已超过{IDLE_TIMEOUT}秒无新任务，准备退出")
                    break
                
                # 从Redis获取任务（阻塞等待，超时时间设为10秒，避免无限阻塞）
                try:
                    queue_name, result_bytes = self.redis.blpop(RESULT_QUEUE, timeout=10)
                except redis.exceptions.ConnectionError:
                    print("⚠️ Redis连接断开，尝试重连...")
                    self.redis = redis.Redis(** self.redis_config)
                    time.sleep(5)
                    continue
                except Exception as e:
                    print(f"⚠️ 获取任务失败：{str(e)[:50]}，重试...")
                    time.sleep(3)
                    continue
                
                # 如果获取到任务，进行处理
                if result_bytes:
                    try:
                        # 解析任务结果
                        result = pickle.loads(result_bytes)
                        task_id = result.get("task_id", f"未知_{int(time.time())}")
                        
                        # 检查任务状态
                        if result.get("status") != "success" or not result.get("result", "").strip():
                            raise ValueError(f"远端执行失败：{result.get('error', '无信息')}")
                        
                        # 获取元信息
                        metadata_bytes = self.redis.hget(self.metadata_key, task_id)
                        if not metadata_bytes:
                            raise ValueError(f"任务{task_id}的元信息不存在")
                        trade_date, stock_code = pickle.loads(metadata_bytes)
                        
                        # 提交到线程池处理
                        self.thread_pool.submit(
                            self._process_single_task,
                            task_id=task_id,
                            csv_str=result["result"],
                            trade_date=trade_date,
                            stock_code=stock_code
                        )
                        
                    except Exception as e:
                        self._update_stats(is_success=False)
                        if self.stats["failed"] % 10 == 0:
                            print(f"❌ 解析任务失败：{str(e)[:80]}（累计失败：{self.stats['failed']}）")
        
        finally:
            # 等待所有线程完成
            self.thread_pool.shutdown(wait=True)
            
            # 最终统计
            total_time = time.time() - self.stats["start_time"]
            total_processed = self.stats["success"] + self.stats["failed"]
            print("\n" + "="*50)
            print("处理结束总结")
            print(f"总运行时间：{int(total_time)}秒")
            print(f"累计处理任务：{total_processed}个")
            print(f"成功：{self.stats['success']}个（{self.stats['success']/total_processed*100:.1f}%）")
            print(f"失败：{self.stats['failed']}个（{self.stats['failed']/total_processed*100:.1f}%）")
            print(f"平均处理速度：{total_processed/total_time:.1f}任务/秒")
            print("="*50)

    def stop(self):
        """停止处理器"""
        self.running = False
        print("ℹ️  收到停止信号，正在处理剩余任务...")

# ---------------------- 主函数 ----------------------
if __name__ == "__main__":
    try:
        # 加载配置
        redis_config = load_redis_config(REDIS_CONFIG_PATH)
        print(f"✅ 加载Redis配置：{redis_config['host']}:{redis_config['port']}")
        
        # 启动动态处理器
        processor = DynamicTaskProcessor(redis_config)
        processor.start_listening()
        
    except Exception as e:
        print(f"❌ 程序启动失败：{str(e)}")
        traceback.print_exc()
        raise SystemExit(1)


✅ 加载Redis配置：220.203.1.124:6379
✅ 存储目录：D:\workspace\xiaoyao\data\stock_minutely_price
✅ Redis连接成功
✅ 初始化线程池（10线程），总任务数：47500
✅ 开始接收并处理结果...
📊 进度：0.0% | 已处理：1/47500 | 成功：0 | 失败：1 | 速度：20.0任务/秒
❌ 任务task_108：失败 → field() got an unexpected keyword argument 'dictionary_encode'
📊 进度：0.0% | 已处理：2/47500 | 成功：0 | 失败：2 | 速度：24.5任务/秒
❌ 任务task_102：失败 → field() got an unexpected keyword argument 'dictionary_encode'
📊 进度：0.0% | 已处理：3/47500 | 成功：0 | 失败：3 | 速度：26.8任务/秒
❌ 任务task_104：失败 → field() got an unexpected keyword argument 'dictionary_encode'
📊 进度：0.0% | 已处理：4/47500 | 成功：0 | 失败：4 | 速度：15.0任务/秒
❌ 任务task_106：失败 → field() got an unexpected keyword argument 'dictionary_encode'
📊 进度：0.0% | 已处理：5/47500 | 成功：0 | 失败：5 | 速度：13.1任务/秒
❌ 任务task_109：失败 → field() got an unexpected keyword argument 'dictionary_encode'
📊 进度：0.0% | 已处理：6/47500 | 成功：0 | 失败：6 | 速度：14.5任务/秒
❌ 任务task_111：失败 → field() got an unexpected keyword argument 'dictionary_encode'
📊 进度：0.0% | 已处理：7/47500 | 成功：0 | 失败：7 | 速度：15.3任务/秒
❌ 任务task_110

KeyboardInterrupt: 